# Datatypes and the Experimentalist Pipeline

The experimentalist pipeline is a tool for running **sequences of functions** (callables) on some data.

If you have $n$ functions $f_i$
$$
f_0(x), f_1(x), ... f_n(x)\, ,
$$

the pipeline of those functions $P_{f_i}$, is defined as
$$
P_{f_i}(x) = f_n(f_{n-1}(...(f_1(f_0(x)))))$\, .
$$

Key principles:
1. The *pipeline passes the data unchanged* between the $f_i$. It is agnostic to the data which the
functions require or return.
2. The $f_i$ can *mutate the data*.
3. The $f_i$ must be *compatible with one another*. It is on the user to ensure this is true.

By being agnostic about the data, the pipeline is extremely simple and flexible. To allow a new kind of data, it is
only required to define the datatype (perhaps as a
[python `dataclass`](https://docs.python.org/3/library/dataclasses.html)) and then start writing $f_i$ which use that
dataclass as an input and output type.

Whilst it is possible to write $f_i$ which change the type of the data which pass through and still work as a
pipeline, it is recommended to return the same type, so $\mathrm{type}(x) = \mathrm{type}(f_i(x))$.

In this tutorial, we'll demonstrate some pipelines which use simple and more complex datatypes, and some pipelines
which have type-mutating $f_i$.